In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

>>Importação das bibliotecas

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

>>Carregamento do dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/AulasPECE/Colab Notebooks/datasets/southeast.csv.zip', nrows=10000)
df.sample(20)

In [ ]:
print("\nInformações sobre o dataset:")
df.info()

>>Análise e Tratamento inicial dos Dados

In [ ]:
print("\nNúmero de valores ausentes em cada coluna:")
print(df.isnull().sum())

In [ ]:
linhas_ausentes = df[df.isnull().any(axis=1)]
linhas_ausentes.head()

In [ ]:
df = df.dropna()

In [ ]:
count_neg9999 = df.isin([-9999]).any(axis=1).sum()
print(f"Número de linhas que contêm valores iguais a -9999 em qualquer coluna: {count_neg9999}")

Número de linhas que contêm valores iguais a -9999 em qualquer coluna: 5207


In [ ]:
df_cleaned = df[~df.isin([-9999]).any(axis=1)]

In [ ]:
print("\nDistribuição de VENTO, RAJADA MAXIMA (m/s)")
sns.histplot(df_cleaned['VENTO, RAJADA MAXIMA (m/s)'])
plt.title('Distribuição VENTO, RAJADA MAXIMA (m/s)')
plt.show()

>>Seleção de Atributos
>>> Investigar a correlação com a coluna 'VENTO, VELOCIDADE HORARIA (m/s)'

In [ ]:
df_numerical = df_cleaned.select_dtypes(include=['float64', 'int64'])
df_numerical = df_numerical.drop(columns=['index'], errors='ignore')

In [ ]:
if 'VENTO, VELOCIDADE HORARIA (m/s)' in df_numerical.columns:
    print("\nCorrelação dos atributos numéricos com 'VENTO, VELOCIDADE HORARIA (m/s)':")
    correlation_with_vento = df_numerical.corr()['VENTO, VELOCIDADE HORARIA (m/s)']
    print(correlation_with_vento)
    columns_above_threshold = correlation_with_vento[correlation_with_vento.abs() > 0.4].index
    df_filtered = df_numerical[columns_above_threshold]
else:
    print("A coluna 'VENTO, VELOCIDADE HORARIA (m/s)' não está presente no DataFrame.")


In [ ]:
print("\nNovo DataFrame com colunas cujas correlações com 'VENTO, VELOCIDADE HORARIA (m/s)' são maiores que 0.4:")
df_filtered.sample(10)

In [ ]:
df_subset = df_numerical.iloc[:, :10]
plt.figure(figsize=(8, 6))
sns.heatmap(df_subset.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlação - Primeiras 10 Colunas')
plt.show()

>> Redutor de Dimensionalidade

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

In [ ]:
pca = PCA(n_components=6)  # Reduzindo para 6 componentes principais
df_numerical_pca = pca.fit_transform(df_numerical)

In [ ]:
print("Variância explicada por cada componente:", pca.explained_variance_ratio_)
print("Variância total explicada:", np.sum(pca.explained_variance_ratio_))

In [ ]:
n_components = df_numerical_pca.shape[1]
df_numerical_pca_df = pd.DataFrame(df_numerical_pca, columns=[f'PC{i+1}' for i in range(n_components)])
plt.figure(figsize=(8, 6))
sns.heatmap(df_numerical_pca_df.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlação - Componentes Principais')
plt.show()

>>Target Encoding

In [ ]:
df_finc = pd.read_csv('/content/PS_20174392719_1491204439457_log.csv.zip', nrows=10000)
df_finc.sample(20)

In [ ]:
mean_encoded = df_finc.groupby('type')['newbalanceDest'].mean()
print(mean_encoded)

In [ ]:
df_finc['type_encoded'] = df_finc['type'].map(mean_encoded)
df_finc.head()

In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(df_finc['type_encoded'], df_finc['newbalanceDest'], color='blue', alpha=0.7)
plt.title('newbalanceDest versus type_encoded')
plt.ylabel('newbalanceDest')
plt.xlabel('type_encoded')
plt.grid(True)
plt.show()

## Exercícios

1. Pré-Processamento de Dados - Tratamento de Valores Ausentes e Inválidos

**Questão**: Considerando que alguns valores ausentes no dataset podem ser substituídos em vez de descartados, quais seriam as possíveis estratégias para substituir valores ausentes e valores inválidos (como -9999)? Implemente ao menos uma estratégia de substituição e compare os resultados com a remoção dos dados.

**Objetivo**: Entender o impacto de diferentes abordagens de tratamento de dados ausentes no dataset

2. Análise de Correlação - Seleção de Atributos

**Questão**: Ao analisar a correlação entre variáveis numéricas e a variável-alvo VENTO, VELOCIDADE HORARIA (m/s), quais atributos apresentam a correlação mais alta? Como essa seleção de atributos poderia influenciar a construção de um modelo preditivo? Justifique a escolha dos atributos a serem mantidos.

**Objetivo**: Identificar os atributos que têm maior influência sobre a variável de interesse, e compreender como a correlação pode ajudar a reduzir a dimensionalidade do problema, mantendo apenas variáveis relevantes.

3. Redução de Dimensionalidade com PCA

**Questão**: Após aplicar PCA ao dataset numérico, os primeiros 6 componentes principais explicam que porcentagem da variância total dos dados? Discuta como a escolha do número de componentes pode impactar o equilíbrio entre a simplificação do modelo e a perda de informação.

**Objetivo:** Analisar a variância explicada por cada componente principal e tomar decisões sobre o número ideal de componentes para representar os dados, considerando a preservação da variabilidade original.

4. Target Encoding em Variáveis Categóricas

**Questão:** Considerando o uso do Target Encoding para a variável type no dataset financeiro, quais seriam as implicações desse método em um modelo preditivo? Como o Target Encoding pode impactar a variabilidade dos dados e o risco de overfitting?

**Objetivo:** Avaliar os prós e contras do Target Encoding, especialmente em relação à sua sensibilidade ao tamanho do dataset e à relação entre as variáveis categóricas e a variável-alvo.

5. Visualização e Interpretação de Dados

**Questão:** A partir da matriz de correlação gerada para as primeiras 10 colunas do dataset, identifique possíveis relações entre variáveis que podem ser exploradas em um modelo preditivo. Como a análise gráfica pode complementar a análise numérica para entender melhor essas relações?

**Objetivo:** Desenvolver a habilidade de interpretar visualizações como mapas de calor de correlação e associar essa interpretação com as decisões de modelagem, como seleção de variáveis e tratamento de multicolinearidade.